In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///./../../app/database/database_test.db"

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [8]:
from typing import List, Dict, Optional, Tuple
from sqlalchemy.orm import Session
from sqlalchemy import text
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

@tool
def search_products_by_category(category_id: int, top_n: int = 5, order_by: Optional[str] = None, order: str = 'ASC', price_range: Optional[Tuple[float, float]] = None) -> str:
# def search_products_by_category(category_id: int, top_n: int = 5, order_by: Optional[str] = None, order: str = 'ASC', price_range: Optional[Tuple[float, float]] = None) -> List[Dict[str, str]]:
    """
    Antes de  ejecutar esta función, debes tener la categoría en la base de datos, si no la tienes debes llamar a la función `get_categories` para obtener las categorías disponibles.
    Busca productos por categoría con opciones de filtrado y ordenamiento, primero debes tener la categoría para hacer la solicitud.

    Args:
        category_id (int): ID de la categoría para buscar productos. Mandatory.
        top_n (int, opcional): Número máximo de productos a devolver. Por defecto es 5.
        order_by (str, opcional): Campo por el cual ordenar los resultados. Por defecto es None.
        order (str, opcional): Orden de los resultados ('ASC' o 'DESC'). Por defecto es 'ASC'.
        price_range (tuple, opcional): Rango de precios (min, max) para filtrar los productos. Por defecto es None.

    Returns:
        str: Tabla con los detalles de los productos.
    """
    session: Session = SessionLocal()
    try:
        query = """
            SELECT p.id, p.name, p.description, p.code, p.price
            FROM products p
            JOIN product_category pc
            ON p.id = pc.product_id
            WHERE pc.category_id = :category_id
        """
        params = {"category_id": category_id}
        
        if price_range:
            query += " AND p.price BETWEEN :min_price AND :max_price"
            params["min_price"] = price_range[0]
            params["max_price"] = price_range[1]
        
        if order_by:
            query += f" ORDER BY {order_by} {order}"
        
        query += f" LIMIT {top_n}"
        
        result = session.execute(text(query), params)
        rows = result.fetchall()
        products = []
        products_str = """
        ID | Nombre | Descripción | Código | Precio
        """
        for row in rows:
            # make a table with the products, id, name, code and price
            products_str += f"{row[0]} | {row[1]} | {row[3]} | {row[4]} \n"

        return products_str
    finally:
        session.close()

In [9]:
from typing import List, Dict
from sqlalchemy.orm import Session
from langchain_core.tools import tool

@tool
def get_categories() -> str:
# def get_categories() -> List[Dict[str, str]]:
    """
    Obtiene todas las categorías de la base de datos.

    Returns:
        str: Tabla con los detalles de las categorías.
    """
    session: Session = SessionLocal()
    try:
        query = "SELECT * FROM categories"
        result = session.execute(query)
        rows = result.fetchall()
        categories = []
        categories_str = ""
        for row in rows:
            # category = {
            #     "id": row[0],
            #     "name": row[1],
            #     "url": row[2]
            # }
            # categories.append(category)
            categories_str += f"{row[0]} - {row[1]}\n"
        return categories_str
    except Exception as e:
        print(e)
    finally:
        session.close()

In [10]:
@tool
def get_product_info_by_search_term(search_term: str) -> str:
    """
    Busca productos por término de búsqueda.
    
    IMPORTANTE: Solo se busca por nombre, descripción y código de producto.

    Args:
        search_term (str): Nombre, descripción o código de producto a buscar.

    Returns:
        str: Lista de productos que coinciden con el término de búsqueda.
    """
    session: Session = SessionLocal()
    try:
        query = """
            SELECT p.id, p.name, p.description, p.code, p.price
            FROM products p
            WHERE p.name LIKE :search_term or p.description LIKE :search_term or p.code LIKE :search_term
        """
        params = {"search_term": f"%{search_term}%"}
        result = session.execute(text(query), params)
        rows = result.fetchall()
        products = []
        products_str = """
        ID | Nombre | Descripción | Código | Precio
        """
        for row in rows:
            # make a table with the products, id, name, code and price
            products_str += f"{row[0]} | {row[1]} | {row[3]} | {row[4]} \n"

        return products_str
    finally:
        session.close()


In [11]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0,
)

llm_with_tools = llm.bind_tools([search_products_by_category])

In [12]:
all_categories = get_categories.invoke({})
messages = [
    SystemMessage(f"""
    Eres un bot especializado en productos de Mikrotik, puedes ayudar a los usuarios a encontrar productos y categorías.

    estas son las categorías disponibles:
    {all_categories}

    """),
    # HumanMessage("Que productos nuevos tiene mikrotik?"),
    HumanMessage("Cuales son los switches más baratos que tiene mikrotik?"),
]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
print(ai_msg.tool_calls)
for tool_call in ai_msg.tool_calls:
    tool_name = tool_call["name"].lower()
    selected_tool = {
        "search_products_by_category": search_products_by_category,
        "get_categories": get_categories,
    }[tool_name]
    # print("first call")
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

result = llm_with_tools.invoke(messages)



[{'name': 'search_products_by_category', 'args': {'category_id': 3, 'order': 'ASC', 'order_by': 'price', 'top_n': 10}, 'id': '4185ad82-25bf-4ee0-af41-32439c8e781c', 'type': 'tool_call'}]


In [13]:
all_categories

'1 - New products\n2 - Ethernet routers\n3 - Switches\n4 - Wireless systems\n5 - Wireless for home and office\n6 - LTE/5G products\n7 - Data over Powerlines\n8 - IoT products\n9 - 60 GHz products\n10 - RouterBOARD\n11 - Enclosures\n12 - Interfaces\n13 - Accessories\n14 - Antennas\n15 - SFP/QSFP\n'

In [14]:
for message in messages:
    print(message)

content='\n    Eres un bot especializado en productos de Mikrotik, puedes ayudar a los usuarios a encontrar productos y categorías.\n\n    estas son las categorías disponibles:\n    1 - New products\n2 - Ethernet routers\n3 - Switches\n4 - Wireless systems\n5 - Wireless for home and office\n6 - LTE/5G products\n7 - Data over Powerlines\n8 - IoT products\n9 - 60 GHz products\n10 - RouterBOARD\n11 - Enclosures\n12 - Interfaces\n13 - Accessories\n14 - Antennas\n15 - SFP/QSFP\n\n\n    ' additional_kwargs={} response_metadata={}
content='Cuales son los switches más baratos que tiene mikrotik?' additional_kwargs={} response_metadata={}
content='' additional_kwargs={} response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-03-11T02:19:35.999266Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1966785000, 'load_duration': 34469667, 'prompt_eval_count': 543, 'prompt_eval_duration': 942000000, 'eval_count': 40, 'eval_duration': 976000000, 'message': Message(role='assistant', 

In [15]:
result.content

'Los switches más baratos que tiene Mikrotik son:\n\n1. **CSS610-8G-2S+IN**: Precio $119.00\n2. **CSS318-16G-2S+IN**: Precio $139.00\n3. **CSS610-1Gi-7R-2S+OUT**: Precio $139.00\n\nEstos switches son ideales para pequeñas redes y dispositivos que requieren una conexión de red estable y confiable.'